# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
from webdriver_manager.chrome import ChromeDriverManager
################ TIME MODLULES ###################
import time
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re

# Scrape [Google Finance](https://www.google.com/finance)

In [3]:
def scrape(company):
    # Initialize lists
    ticker=[]
    listed=[]
    public=[]
    # open chrome driver
    #driver=webdriver.Chrome(executable_path="C:\\Users\\Amit\\Desktop\\Internships\\BiosectRx\\Task_1\\chromedriver.exe")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    for c,i in enumerate(company):
        print(c,i)
        try:
            driver.get('https://www.google.com/search?tbm=fin&q=NASDAQ%3A%20{}'.format("+".join(i.split()))) # search company as NASDAQ listed
            source = driver.page_source
            soup=bs4.BeautifulSoup(source, 'html.parser')
            ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0]) # get ticker (up or down)
            listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0]) # where is the company listed?
            public.append(1) # company is public if on NASDAQ
        except:
            time.sleep(5)
            try:
                driver.get('https://www.google.com/search?tbm=fin&q={}'.format("+".join(i.split()))) # search company without NASDAQ
                source = driver.page_source
                soup=bs4.BeautifulSoup(source, 'html.parser')
                ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0]) # get ticker (up or down)
                listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0]) # where is the company listed?
                public.append(1) # company is public if on NYSE or SHA etc.
            except:
                # if company is not on NASDAQ, NYSE, SHA etc., consider it non-public
                ticker.append(np.nan) 
                listed.append(np.nan)
                public.append(0)
        time.sleep(5) # wait for 5 sec after every loop
    driver.close() # close driver
    return public,listed,ticker

In [6]:
company=list(pd.read_excel('companies.xlsx')['Companies']) # read list of companies
public,listed,ticker=scrape(company) # scrape google finance
df=pd.DataFrame(zip(company,public,listed,ticker),columns=['Company','Public','Listed On','Ticker']) # create dataframe
company2=df.loc[(df['Public']==1)]['Company']
public,listed,ticker=scrape(company2) # update dataframe
df2=pd.DataFrame(zip(company2,public,listed,ticker),columns=['Company','Public','Listed On','Ticker'])
df2.index=list(df.loc[(df['Public']==1)].index)
df.update(df2)
df['Public']=df['Public'].astype(int)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
 
[WDM] - Driver [C:\Users\Lenovo\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
0 Athena Bioscience
1 miniPCR bio
2 Ensysce Biosciences
3 VEO Ophthalmics
4 Disc Medicine
5 SpyBiotech
6 Jaan Biotherapeutics
7 S2 Genomics
8 Mycenax Biotech
9 Vigilant Biosciences
10 KitCheck
11 ReCode Therapeutics
12 Parvizi Surgical Innovation
13 Coracoid Solutions
14 Cleave Therapeutics
15 Spruce Biosciences
16 HalioDx
17 Arctoris Ltd
18 MedeAnalytics
19 Biotalys NV
20 Haoma Medica
21 Ashvattha Therapeutics
22 Terns Pharmaceuticals
23 Reneo Pharmaceuticals
24 KromaTiD
25 Cend Therapeutics
26 Jennewein Biotechnologie GmbH
27 TransMed7
28 Life Spine
29 Oculis S.A.
30 Ichnos Sciences
31 Congenica
32 Aural Analytics, Inc.
33 ERS Genomics
34 Aziyo Biologics
35 IZI Medical Products, LLC
36 Curi Bio
37 Sirtex Medical
38 Tikomed AB
39 LuminUltra
40 89bio
41 Immunity Pharma
42 Fibral

In [7]:
df1=df.loc[(df['Public']==0)] # filter out non-public companies
df2=df.loc[(df['Public']==1)] # filter out public companies
df.shape,df1.shape

((759, 4), (759, 4))

In [8]:
# Only 19/451 companies are public, rest 432 are non-public
# public companies are the following
df2

,Company,Public,Listed On,Ticker


In [9]:
# save to excel sheet
df1.to_excel('non-public.xlsx',index=False)
df2.to_excel('public.xlsx',index=False)